**This Jupyter Notebook requires a Julia 1.10 Kernel**

In [1]:
using Pkg

In [3]:
#Pkg.activate("../../../../NN-Safety-via-dL/repos/NCubeV-Reinstall/")
Pkg.activate(".")

  Activating project at `~/Dokumente/Projects/CPS/ABZ/abz2025-case-study/versaille/analysis`


In [4]:
python_env_loc = readlines("pythonenv")[1]
println("Assuming python environment is at $python_env_loc")

Assuming python environment is at /home/samuel/anaconda3/envs/abz-env


In [5]:
ENV["PYTHON"] = python_env_loc

"/home/samuel/anaconda3/envs/abz-env"

In [6]:
try
    using SNNT
catch
end

In [7]:
using SNNT
using JLD
using Glob
using Plots
using PyCall

In [8]:
function load_stars(folder, prefix, limit)
    println("Loading results from $folder/$prefix-*.jld")
    stars = []
    for file in glob("$folder/$prefix-*.jld")
        if occursin("summary",file)
            continue
        end
        println("Loading $file")
        res = load(file)
        for res in res["result"]
            append!(stars, res.stars)
        end
        if length(stars) > limit
            break
        end
    end
    return stars
end

load_stars (generic function with 1 method)

In [9]:
function summarize_and_load(folder, prefix)
    println("Loading results from $folder/$prefix-*.jld")
    results = []
    metadata = nothing
    for file in glob("$prefix-*.jld",folder)
        println(file)
        if occursin("summary",file)
            continue
        end
        cur_results = load(file)
        if haskey(cur_results,"backup_meta")
            metadata = cur_results["backup_meta"]
        end
        append!(results,cur_results["result"])
    end
    result_summary = SNNT.VerifierInterface.reduce_results(results)
    save("$folder/$prefix-summary.jld","result",result_summary,"args",metadata)
    return (result_summary, metadata)
end

summarize_and_load (generic function with 1 method)

In [10]:
results = load_stars("../results", "improved-ld-all-cars",Inf)
nothing

Loading results from ../results/improved-ld-all-cars-*.jld
Loading ../results/improved-ld-all-cars-112.jld
Loading ../results/improved-ld-all-cars-118.jld
Loading ../results/improved-ld-all-cars-121.jld
Loading ../results/improved-ld-all-cars-161.jld
Loading ../results/improved-ld-all-cars-209.jld
Loading ../results/improved-ld-all-cars-224.jld
Loading ../results/improved-ld-all-cars-237.jld
Loading ../results/improved-ld-all-cars-245.jld
Loading ../results/improved-ld-all-cars-260.jld
Loading ../results/improved-ld-all-cars-266.jld
Loading ../results/improved-ld-all-cars-312.jld
Loading ../results/improved-ld-all-cars-315.jld
Loading ../results/improved-ld-all-cars-325.jld
Loading ../results/improved-ld-all-cars-330.jld
Loading ../results/improved-ld-all-cars-36.jld
Loading ../results/improved-ld-all-cars-377.jld
Loading ../results/improved-ld-all-cars-409.jld
Loading ../results/improved-ld-all-cars-44.jld
Loading ../results/improved-ld-all-cars-463.jld
Loading ../results/improved-ld-

In [11]:
println("Total Stars Retrieved: ",length(results))
# Filtering out counterexampels where front car position is too close due to float rounding
# This is a problem with the counterexample extraction from Z3 *not* with the counterexamples themselves

function dl_invariant(cex)
    xf = 5*40*cex[2]
    vf = 2*40*cex[4]
    xl = 5*40*(cex[2]+cex[7])
    vl = 2*40*(cex[4]+cex[9])
    return ((vl >= 0) & (vf >= 0) & (vl <= 40.0) & (vf <= 40.0) & (xf + 5.0 <= xl) & (xf + vf^2/(2*5.0) + 5.0 < xl + vl^2/(2*5.0)))
end

results_float = filter(x -> !(dl_invariant(x.counter_example[1])), results)
results = filter(x -> dl_invariant(x.counter_example[1]), results)
println("Total Stars After Filtering: ",length(results))

Total Stars Retrieved: 11059
Total Stars After Filtering: 10937


In [12]:
pushfirst!(pyimport("sys")."path", "");

In [13]:
pyimport("gymnasium")

PyObject <module 'gymnasium' from '/home/samuel/anaconda3/envs/abz/lib/python3.10/site-packages/gymnasium/__init__.py'>

In [14]:
py"""
from highway_reproduction import *
from custom_highway import *

# Acceleration control bug: Must be >0 to avoid car accidentally going backwards...
BrakingVehicle.MIN_SPEED = 1.0

from highway_env.vehicle.kinematics import Vehicle
from highway_env.vehicle.behavior import IDMVehicle
Vehicle.MIN_SPEED = 0.0
IDMVehicle.ACC_MAX = 5.0
IDMVehicle.MIN_SPEED = 0.0

envs_all = reproduce_crash_get_envs(None,worst_case_braking=-5.0,weak_brake=False)
envs_all = [env for _,env in envs_all]
model = load_model("../nets/nn_small_improved-ld.zip")
envs_worst_case = [envs_all[1]]

def invariant(xf, vf, xl, vl):
    return vl >= 0 and vf >= 0 and vl <= 40.0 and vf <= 40.0 and xf + 5.0 <= xl and xf + vf**2/(2*5.0) + 5.0 < xl + vl**2/(2*5.0)

def quant_invariant(xf, vf, xl, vl):
    return max(
        -vl,
        -vf,
        vl - 40.0,
        vf - 40.0,
        xf + 5.0-xl,
        xf + vf**2/(2*5.0) + 5.0 - xl - vl**2/(2*5.0)
    )

def get_results(observation):
    global envs_all
    global model
    set_observation_seed(envs_all, observation)
    results = reproduce_crash_evaluate(envs_all, model, test_runs=1, default_action=0)
    return results

def get_results_wc(observation):
    global envs_worst_case
    global model
    set_observation_seed(envs_worst_case, observation)
    results = reproduce_crash_evaluate(envs_worst_case, model, test_runs=1, default_action=0)
    return results
"""
get_results = py"get_results"
get_results_wc = py"get_results_wc"
invariant = py"invariant"
quant_invariant = py"quant_invariant"

/home/samuel/anaconda3/envs/abz/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
/home/samuel/anaconda3/envs/abz/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
/home/samuel/anaconda3/envs/abz/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implic

PyObject <function quant_invariant at 0x74b275b272e0>

In [15]:
crash_results = []
@time for star in results[1:100]
    obs = star.counter_example[1]
    #in_inv = invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9]))
    #print(in_inv)
    push!(crash_results,get_results(star.counter_example[1]))
end

 69.775201 seconds (334.78 k allocations: 22.311 MiB, 0.37% compilation time)


In [19]:
crash_results = []
@time for star in results
    obs = star.counter_example[1]
    #in_inv = invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9]))
    #print(in_inv)
    push!(crash_results,get_results(star.counter_example[1]))
end

6918.145578 seconds (1.69 M allocations: 64.069 MiB, 0.00% gc time)


In [20]:
save("improved-full-crash-results.jld","crash_results",crash_results)

In [21]:
function summarize_crashes(crash_results)
    default_env_nn = 0
    default_env_nn_rew = []
    front_brake_env_nn = 0
    front_brake_env_nn_rew = []
    default_env_brake = 0
    default_env_brake_rew = []
    front_brake_env_brake = 0
    front_brake_env_brake_rew = []
    for res in crash_results
        if res[1][1][1] == 1
            default_env_nn += 1
        end
        if res[1][1][2] == 1
            front_brake_env_nn += 1
        end
        if res[2][1][1] == 1
            default_env_brake += 1
        end
        if res[2][1][2] == 1
            front_brake_env_brake += 1
        end
        push!(default_env_nn_rew,res[1][2][1])
        push!(front_brake_env_nn_rew,res[1][2][2])
        push!(default_env_brake_rew,res[2][2][1])
        push!(front_brake_env_brake_rew,res[2][2][2])
    end
    println("Crashes:")
    println("Default Env NN:\t\t\t",default_env_nn)
    println("Front Brake Env NN:\t\t",front_brake_env_nn)
    println("Default Env Brake:\t\t",default_env_brake)
    println("Front Brake Env Brake:\t\t",front_brake_env_brake)
    return (default_env_nn_rew, front_brake_env_nn_rew, default_env_brake_rew, front_brake_env_brake_rew)
end

summarize_crashes (generic function with 1 method)

In [22]:
crash_results = load("improved-full-crash-results.jld")["crash_results"]

10937-element Vector{Any}:
 (([0, 0], [22.875, 16.872500000000002]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [21.9375, 16.875]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [22.0625, 16.9325]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [22.8125, 17.125]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [21.75, 17.1125]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [21.1875, 17.1125]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [20.6875, 17.110000000000003]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [21.8125, 17.1125]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [21.1875, 17.1125]), ([0, 0], [16.3125, 16.3125]))
 (([0, 0], [21.625, 17.0625]), ([0, 0], [16.3125, 16.3125]))
 ⋮
 (([0, 1], [20.007499999999997, 1.46]), ([0, 0], [15.125, 15.125]))
 (([0, 1], [19.71875, 1.46]), ([0, 0], [15.125, 15.125]))
 (([0, 1], [19.862499999999997, 1.46]), ([0, 0], [15.125, 15.125]))
 (([0, 1], [17.816875000000003, 0.71875]), ([0, 0], [15.0, 15.0]))
 (([0, 1], [17.125, 0.681]), ([0, 0], [15.0, 15.0]))
 (([0, 1], [17.3125,

In [23]:
summarize_crashes(crash_results)

Crashes:
Default Env NN:			4852
Front Brake Env NN:		8713
Default Env Brake:		181
Front Brake Env Brake:		40


(Any[22.875, 21.9375, 22.0625, 22.8125, 21.75, 21.1875, 20.6875, 21.8125, 21.1875, 21.625  …  19.8125, 20.007499999999997, 19.71875, 19.862499999999997, 17.816875000000003, 17.125, 17.3125, 18.0625, 18.40625, 18.09375], Any[16.872500000000002, 16.875, 16.9325, 17.125, 17.1125, 17.1125, 17.110000000000003, 17.1125, 17.1125, 17.0625  …  1.46, 1.46, 1.46, 1.46, 0.71875, 0.681, 0.7050000000000001, 0.75625, 1.37138, 1.37138], Any[16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125  …  15.125, 15.125, 15.125, 15.125, 15.0, 15.0, 15.0, 15.03125, 15.03125, 15.03125], Any[16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125, 16.3125  …  15.125, 15.125, 15.125, 15.125, 15.0, 15.0, 15.0, 15.03125, 15.03125, 15.03125])

In [24]:
weird_cases = findall(map(x->x[2][1][1]==1 || x[2][1][2]==1,crash_results))
println(weird_cases)
nothing

[123, 140, 147, 150, 152, 164, 168, 173, 174, 280, 285, 296, 297, 610, 615, 623, 657, 661, 663, 666, 670, 672, 678, 708, 709, 717, 719, 722, 724, 726, 748, 751, 797, 803, 840, 843, 860, 892, 903, 906, 910, 912, 1148, 1185, 1235, 1258, 1298, 1687, 1724, 1726, 1730, 1737, 1740, 1741, 1800, 1801, 1802, 1804, 1809, 1810, 1811, 1812, 1813, 2163, 2165, 2166, 2167, 2168, 2169, 2170, 2171, 2172, 2173, 2209, 2333, 2335, 2336, 2337, 2338, 2339, 2340, 2358, 2359, 2360, 2379, 2481, 2707, 2743, 2988, 2989, 2990, 2993, 2995, 2996, 2997, 2998, 2999, 3013, 3497, 3668, 3701, 3708, 3709, 3749, 3874, 3883, 3890, 3997, 4044, 4081, 4082, 4137, 4255, 4263, 4441, 4472, 4487, 4586, 4599, 4971, 4972, 4973, 4988, 4999, 5006, 5007, 5172, 5303, 6039, 6484, 6500, 6520, 6542, 6559, 6560, 6586, 6614, 6620, 6756, 6783, 6892, 6904, 6908, 6917, 7102, 7123, 7158, 7184, 7298, 7322, 7363, 7402, 7442, 7460, 7468, 7494, 7521, 7555, 7596, 7618, 7654, 7670, 7771, 7772, 7791, 7795, 7886, 7989, 8026, 8056, 8060, 8067, 8085, 808

In [25]:
summarize_crashes(crash_results[weird_cases])

Crashes:
Default Env NN:			203
Front Brake Env NN:		201
Default Env Brake:		181
Front Brake Env Brake:		40


(Any[2.916246875, 2.92984296875, 3.0868125, 2.962425, 2.94870625, 2.92984296875, 3.0868125, 2.962425, 2.94870625, 3.1595  …  1.0225, 1.0207421875, 1.0172265624999999, 1.0084374999999999, 0.94365625, 1.0154687500000001, 0.94365625, 1.1486999999999998, 1.1776550000000001, 2.116866], Any[2.950296875, 2.92984296875, 3.1634375, 3.019625, 2.94870625, 2.92984296875, 3.1634375, 3.019625, 2.94870625, 3.2575  …  1.0625, 1.060546875, 1.0172265624999999, 1.046875, 0.94365625, 1.0546875, 0.94365625, 1.1486999999999998, 1.1776550000000001, 1.2984], Any[3.685390625, 3.66393828125, 4.3571375, 4.23283, 3.69671875, 3.66393828125, 4.3571375, 4.23283, 3.69671875, 4.42875  …  1.89548, 1.8907703125, 1.3662109375, 1.8667625, 1.29296875, 1.87678125, 1.29296875, 16.3125, 16.640625, 16.53125], Any[16.18359375, 16.1630859375, 16.33984375, 16.21875, 16.1953125, 16.1630859375, 16.33984375, 16.21875, 16.1953125, 16.421875  …  15.375, 15.3720703125, 15.3662109375, 15.3515625, 15.29296875, 15.36328125, 15.29296875, 4

There are some cases where even braking with full force (provably correct!) leads to a crash.

Hypothesis for Explanation:
- This is an artefact of `highway-env`'s too imprecise euler approximation

In [26]:
py"""
for env in envs_all:
    env.unwrapped.config["simulation_frequency"] = 100
"""

In [27]:
crash_results_weird = []
crash_result_weird_indices = []
@time for (i,star) in enumerate(results[weird_cases]) #[1:100]
    obs = star.counter_example[1]
    in_inv = invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9]))
    if in_inv
        push!(crash_results_weird,get_results(star.counter_example[1]))
        push!(crash_result_weird_indices, weird_cases[i])
    else
        println("Just outside inv: ", quant_invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9])))
    end
end

395.331856 seconds (59.16 k allocations: 2.794 MiB, 0.01% compilation time)


In [28]:
summarize_crashes(crash_results_weird)

Crashes:
Default Env NN:			197
Front Brake Env NN:		200
Default Env Brake:		20
Front Brake Env Brake:		0


(Any[2.920612931144571, 2.9139917544437552, 3.050045252107387, 2.954661399072105, 2.950253494093824, 2.9139917544437552, 3.050045252107387, 2.954661399072105, 2.9303493890414853, 3.1457722116079423  …  1.0239300679909245, 1.0221706631183278, 1.0123383142918976, 1.0067540968388577, 0.9471276402350541, 1.0184903958404805, 0.9471276402350541, 1.1326545449637822, 0.484375, 1.4050935139138276], Any[2.9322412188501086, 2.9313454151147447, 3.108665893357351, 2.988574965616058, 2.962082253784607, 2.9313454151147447, 3.108665893357351, 2.988574965616058, 2.962082253784607, 3.1863374214857734  …  1.032192069775687, 1.0270695405702308, 1.0202577082117903, 1.017794076077375, 0.9495884629880558, 1.0249930729265309, 0.9495884629880558, 1.1326545449637822, 0.4514691528924482, 1.2646565304687143], Any[16.18359374999999, 16.163085937499986, 16.339843749999996, 16.218749999999993, 16.19531249999999, 16.163085937499986, 16.339843749999996, 16.218749999999993, 16.19531249999999, 16.421875000000004  …  15.

In [29]:
py"""
for env in envs_all:
    env.unwrapped.config["simulation_frequency"] = 1000
"""

In [30]:
weird_cases2 = findall(map(x->x[2][1][1]==1 || x[2][1][2]==1,crash_results_weird))
println(weird_cases2)
crash_result_weird_indices = weird_cases[weird_cases2]
println(crash_result_weird_indices)
nothing

[11, 14, 20, 27, 32, 35, 38, 59, 89, 93, 98, 110, 111, 121, 157, 161, 179, 180, 195, 207]
[285, 610, 666, 719, 751, 840, 892, 1809, 2988, 2995, 3013, 4081, 4082, 4972, 7521, 7654, 8256, 8266, 8569, 8693]


In [31]:
crash_results_weird2 = []
crash_result_weird_indices2 = []
@time for (i,star) in enumerate(results[crash_result_weird_indices]) #[1:100]
    obs = star.counter_example[1]
    in_inv = invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9]))
    if in_inv
        push!(crash_results_weird2,get_results(star.counter_example[1]))
        push!(crash_result_weird_indices2, weird_cases[i])
    else
        println("Just outside inv: ", quant_invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9])))
    end
end

320.514486 seconds (3.71 k allocations: 139.266 KiB)


In [32]:
summarize_crashes(crash_results_weird2)

Crashes:
Default Env NN:			20
Front Brake Env NN:		17
Default Env Brake:		2
Front Brake Env Brake:		0


(Any[3.18007978518201, 0.030813011792178158, 0.030689798214003083, 0.030689798214003083, 0.030813011792178158, 0.029783420046338882, 0.029783420046338882, 1.9345414354429031, 3.2392610867213496, 1.941186824954603, 1.9908507234105044, 8.877732746843126, 8.877734001884285, 3.9438407684970405, 0.8205261629190646, 0.8203970302655859, 1.0872136350792845, 1.102077832252056, 1.034108687563936, 1.0121131543306392], Any[3.1958606476860916, 0.03156010778272078, 0.03238476185499062, 0.03238476185499062, 0.03156010778272078, 0.03119677384005981, 0.03119677384005981, 1.9510194821729816, 6.368868206649897, 1.9552756031878533, 2.0064581339928838, 15.379577636718654, 15.379577636718654, 15.966796875000647, 0.822180441692178, 0.8221170134004525, 1.1001158786542744, 1.7132249823044665, 1.041898479892962, 1.0196580580553665], Any[16.32617187500003, 15.0, 15.0, 15.0, 15.0, 0.00011401154139095704, 0.0008302872111063953, 15.722656250000078, 15.729980468750078, 15.730285644531328, 15.782470703125085, 15.3756

In [34]:
weird_cases3 = findall(map(x->x[2][1][1]==1 || x[2][1][2]==1,crash_results_weird2))
println(weird_cases3)
crash_result_weird_indices2 = weird_cases[weird_cases3]
println(crash_result_weird_indices2)
nothing

[6, 7]
[164, 168]


In [35]:
py"""
for env in envs_all:
    env.unwrapped.config["simulation_frequency"] = 10000
"""

In [36]:
crash_results_weird3 = []
crash_result_weird_indices3 = []
@time for (i,star) in enumerate(results[crash_result_weird_indices2]) #[1:100]
    obs = star.counter_example[1]
    in_inv = invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9]))
    if in_inv
        push!(crash_results_weird3,get_results(star.counter_example[1]))
        push!(crash_result_weird_indices3, weird_cases[i])
    else
        println("Just outside inv: ", quant_invariant(5*40*obs[2], 2*40*obs[4], 5*40*(obs[2]+obs[7]), 2*40*(obs[4]+obs[9])))
    end
end

316.388912 seconds (374 allocations: 14.047 KiB)


In [37]:
summarize_crashes(crash_results_weird3)

Crashes:
Default Env NN:			2
Front Brake Env NN:		2
Default Env Brake:		0
Front Brake Env Brake:		0


(Any[2.9140140973448396, 3.0502231354415965], Any[2.930820012953526, 3.10632578223569], Any[16.163085937501556, 16.339843750000764], Any[16.163085937501556, 16.339843750000764])

In [38]:
println(results[crash_result_weird_indices[1]].counter_example[1])

[1.0, 0.9999998494517268, 0.0, 0.439453125, 0.0, 1.0, 0.6435546875, 0.0, -0.4375, 0.0, 1.0, 0.6689453125, 0.0, -0.435546875, 0.0, 1.0, 0.974853515625, 0.0, 0.4453125, 0.0, 1.0, 0.9999980926513672, 0.0, 0.9999990463256836, 0.0]
